# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig

# -------------------------------------------------------
# Global test constants
# -------------------------------------------------------

USER_ID = "demo-user-planner"
SESSION_ID = "sess_planner_only_001"

# -------------------------------------------------------
# Build LLM adapter and RuntimeConfig
# IMPORTANT: Replace build_llm_adapter() with your actual builder.
# This must be the same adapter/config used in your runtime.
# -------------------------------------------------------

llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=True,
    enable_websearch=True,
    tools_mode="auto",
    enable_user_longterm_memory=True,    
)

# Hard-coded intent

In [ ]:
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.planning.stepplan_models import EngineHints, PlanIntent, StepAction

sp = StepPlanner(StepPlannerConfig())

# TEST 2: freshness -> websearch
plan = sp.build_from_hints(
    user_message="What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates.",
    engine_hints=EngineHints(
        enable_websearch=True,
        enable_ltm=True,
        enable_rag=False,
        enable_tools=False,
        intent=PlanIntent.FRESHNESS,
    ),
)
assert plan.intent == PlanIntent.FRESHNESS
assert plan.steps[0].action == StepAction.USE_WEBSEARCH
print("OK freshness routing:", plan.intent, [s.action.value for s in plan.steps])

# TEST 3: project -> ltm
plan = sp.build_from_hints(
    user_message="StepPlanner vs EnginePlanner integration in Intergrax. What should own intent?",
    engine_hints=EngineHints(
        enable_websearch=True,
        enable_ltm=True,
        enable_rag=False,
        enable_tools=False,
        intent=PlanIntent.PROJECT_ARCHITECTURE,
    ),
)
assert plan.intent == PlanIntent.PROJECT_ARCHITECTURE
assert plan.steps[0].action == StepAction.USE_USER_LONGTERM_MEMORY_SEARCH
print("OK project routing:", plan.intent, [s.action.value for s in plan.steps])

OK freshness routing: PlanIntent.FRESHNESS ['USE_WEBSEARCH', 'SYNTHESIZE_DRAFT', 'VERIFY_ANSWER', 'FINALIZE_ANSWER']
OK project routing: PlanIntent.PROJECT_ARCHITECTURE ['USE_USER_LONGTERM_MEMORY_SEARCH', 'SYNTHESIZE_DRAFT', 'VERIFY_ANSWER', 'FINALIZE_ANSWER']


In [ ]:
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.planning.stepplan_models import EngineHints


sp = StepPlanner(StepPlannerConfig())

tests = [
    "Explain how to implement an async retry strategy in Python for API calls.",
    "What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates.",
    "In my Intergrax runtime, should StepPlanner be separate from EnginePlanner? Give recommendation based on our architecture decisions.",
]

for i, q in enumerate(tests, 1):
    plan = sp.build_from_hints(user_message=q, engine_hints=EngineHints(
        enable_websearch=True,
        enable_ltm=True,
        enable_rag=False,
        enable_tools=False,
    ))
    print("=" * 80)
    print(f"TEST {i}: {q}")
    print("plan_id:", plan.plan_id)
    print("mode:", plan.mode)
    print("intent:", plan.intent)
    print("steps:", [s.action for s in plan.steps])
    print("last:", plan.steps[-1].action)


TEST 1: Explain how to implement an async retry strategy in Python for API calls.
plan_id: stepplan-001
mode: PlanMode.EXECUTE
intent: PlanIntent.GENERIC
steps: [<StepAction.SYNTHESIZE_DRAFT: 'SYNTHESIZE_DRAFT'>, <StepAction.VERIFY_ANSWER: 'VERIFY_ANSWER'>, <StepAction.FINALIZE_ANSWER: 'FINALIZE_ANSWER'>]
last: StepAction.FINALIZE_ANSWER
TEST 2: What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates.
plan_id: stepplan-001
mode: PlanMode.EXECUTE
intent: PlanIntent.GENERIC
steps: [<StepAction.SYNTHESIZE_DRAFT: 'SYNTHESIZE_DRAFT'>, <StepAction.VERIFY_ANSWER: 'VERIFY_ANSWER'>, <StepAction.FINALIZE_ANSWER: 'FINALIZE_ANSWER'>]
last: StepAction.FINALIZE_ANSWER
TEST 3: In my Intergrax runtime, should StepPlanner be separate from EnginePlanner? Give recommendation based on our architecture decisions.
plan_id: stepplan-001
mode: PlanMode.EXECUTE
intent: PlanIntent.GENERIC
steps: [<StepAction.SYNTHESIZE_DRAFT: 'SYNTHESIZE_DRAFT'>, <StepAction.VERIFY_ANS

### EnginePlanner → StepPlanner adapter test (`build_plan_from_engine_plan`)

This test validates the integration point between the LLM-based EnginePlanner and the deterministic StepPlanner.

Flow:
1. Build a `RuntimeRequest` and `RuntimeState` (capabilities on/off).
2. Call `EnginePlanner.plan(...)` and capture `EnginePlan`.
3. Pass that `EnginePlan` into `StepPlanner.build_plan_from_engine_plan(...)`.
4. Inspect `ExecutionPlan`:
   - If `intent == CLARIFY`: ensure StepPlanner uses `EnginePlan.clarifying_question`.
   - Otherwise: ensure the step sequence is consistent with enabled flags (websearch/rag/tools/ltm).


In [5]:
from intergrax.runtime.nexus.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.nexus.planning.stepplan_models import EngineHints, ExecutionPlan
from intergrax.runtime.nexus.planning.engine_plan_models import EnginePlan
from intergrax.runtime.nexus.planning.plan_builder_helper import build_plan


def engine_plan_to_hints(plan: EnginePlan) -> EngineHints:
    return EngineHints(
        enable_websearch=bool(plan.use_websearch),
        enable_ltm=bool(plan.use_user_longterm_memory),
        enable_rag=bool(plan.use_rag),
        enable_tools=bool(plan.use_tools),
        intent=plan.intent,
    )


def print_engine_plan(plan: EnginePlan) -> None:
    print("=== ENGINE PLAN ===")
    print(f"intent                : {plan.intent}")
    print(f"next_step             : {plan.next_step}")
    print(f"ask_clarify           : {plan.ask_clarifying_question}")
    print(f"clarifying_question   : {plan.clarifying_question}")
    print(f"use_websearch         : {plan.use_websearch}")
    print(f"use_tools             : {plan.use_tools}")
    print(f"use_rag               : {plan.use_rag}")
    print(f"use_user_ltm          : {plan.use_user_longterm_memory}")
    if plan.reasoning_summary:
        print(f"reasoning_summary     : {plan.reasoning_summary}")
    print()


def print_execution_plan(ep: ExecutionPlan) -> None:
    print("=== EXECUTION PLAN ===")
    print(f"plan_id   : {ep.plan_id}")
    print(f"mode      : {ep.mode}")
    print(f"intent    : {ep.intent}")
    print("steps:")
    for i, st in enumerate(ep.steps, start=1):
        print(f"  {i:02d}. id={st.step_id} action={st.action} expected={st.expected_output_type}")
    print()


In [ ]:
USER_ID = "nb-test-user"
SESSION_ID = "nb-engine-to-step-001"

msg = "What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates."

engine_plan = await build_plan(
    config=config,
    message=msg,
    user_id=USER_ID,
    session_id=SESSION_ID,
    run_id="nb-run-freshness-001",
    instructions="Use a concise technical style. Never use emojis in code blocks or technical documentation.",
    attachments=[],
    base_history=[],
)

print_engine_plan(engine_plan)

step_planner = StepPlanner(cfg=StepPlannerConfig())
hints = engine_plan_to_hints(engine_plan)

execution_plan = step_planner.build_from_hints(
    user_message=msg,
    engine_hints=hints,
    plan_id="stepplan-freshness-001",
)

print_execution_plan(execution_plan)


=== ENGINE PLAN ===
intent                : PlanIntent.FRESHNESS
next_step             : EngineNextStep.WEBSEARCH
ask_clarify           : False
clarifying_question   : None
use_websearch         : True
use_tools             : False
use_rag               : False
use_user_ltm          : False
reasoning_summary     : User's request requires up-to-date information on OpenAI Responses API and tool calling, so websearch is the most suitable next step.

=== EXECUTION PLAN ===
plan_id   : stepplan-freshness-001
mode      : PlanMode.EXECUTE
intent    : PlanIntent.FRESHNESS
steps:
  01. id=StepId.WEBSEARCH action=StepAction.USE_WEBSEARCH expected=ExpectedOutputType.SEARCH_RESULTS
  02. id=StepId.DRAFT action=StepAction.SYNTHESIZE_DRAFT expected=ExpectedOutputType.DRAFT
  03. id=StepId.VERIFY action=StepAction.VERIFY_ANSWER expected=ExpectedOutputType.VERIFIED
  04. id=StepId.FINAL action=StepAction.FINALIZE_ANSWER expected=ExpectedOutputType.FINAL



In [ ]:
msg = "What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates."

engine_plan = await build_plan(
    config=config,
    message=msg,
    user_id=USER_ID,
    session_id=SESSION_ID,
    run_id="nb-run-freshness-002",
    instructions="Use a concise technical style. Never use emojis in code blocks or technical documentation.",
    attachments=[],
    base_history=[],
)

print_engine_plan(engine_plan)

step_planner = StepPlanner(cfg=StepPlannerConfig())

execution_plan = step_planner.build_from_engine_plan(
    user_message=msg,
    engine_plan=engine_plan,
    plan_id="stepplan-freshness-002",
)

print_execution_plan(execution_plan)


=== ENGINE PLAN ===
intent                : PlanIntent.FRESHNESS
next_step             : EngineNextStep.WEBSEARCH
ask_clarify           : False
clarifying_question   : None
use_websearch         : True
use_tools             : False
use_rag               : False
use_user_ltm          : False
reasoning_summary     : User request requires up-to-date information; using websearch for freshness.

=== EXECUTION PLAN ===
plan_id   : stepplan-freshness-002
mode      : PlanMode.EXECUTE
intent    : PlanIntent.FRESHNESS
steps:
  01. id=StepId.WEBSEARCH action=StepAction.USE_WEBSEARCH expected=ExpectedOutputType.SEARCH_RESULTS
  02. id=StepId.DRAFT action=StepAction.SYNTHESIZE_DRAFT expected=ExpectedOutputType.DRAFT
  03. id=StepId.VERIFY action=StepAction.VERIFY_ANSWER expected=ExpectedOutputType.VERIFIED
  04. id=StepId.FINAL action=StepAction.FINALIZE_ANSWER expected=ExpectedOutputType.FINAL

